In [1]:
##mew
import cv2
from final_function import BotFinder
import time
import math
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
from StartEnd import Start_End
global boxes,sorted_keys
boxes = {}
sorted_keys = []

In [2]:
def size_mod(img,times):
    h,w = img.shape[:2]
    dim = (int(w/times), int(h/times))
    return cv2.resize(img, dim)

def min_dist(coord_1,coord_2):
    dist = math.sqrt(  (coord_1[0] - coord_2[0])**2  + (coord_1[1] - coord_2[1])**2  )
    return dist

feed = cv2.imread(f'C:/Users/OHM/Desktop/pytthon/error_imgs/26.jpg')

In [3]:
hsv = cv2.cvtColor(feed, cv2.COLOR_BGR2HSV)
lower_blue = np.array([30,90,80])
upper_blue = np.array([50,255,255])
# coord = start_end_coord(hsv,lower_blue ,upper_blue )
# print(f'1 --> start point: {coord[0]} || end point: {coord[1]}')
height,width = feed.shape[:2]

#dim = (int(w/2), int(h/2))
#feed = cv2.resize(feed, dim)
s = time.time()
gray = cv2.cvtColor(feed, cv2.COLOR_BGR2GRAY)
hls = cv2.cvtColor(feed, cv2.COLOR_BGR2HLS)
mask = cv2.inRange(hls, (0,100,0), (255,255,255))
blur = cv2.GaussianBlur(mask, (5,5), 0)
blur = cv2.GaussianBlur(blur, (5,5), 0)
kernel = np.ones((5,5), np.uint8)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
kernel = np.ones((13,13), np.uint8)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
kernel = np.ones((3,3), np.uint8)
img_erosion = cv2.erode(closing, kernel, iterations=1)

#img_dilation = cv2.dilate(blur, kernel, iterations=5)
#print("dilated inage")
#
contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
area = feed.shape[:2][1]*feed.shape[:2][0]
#min_area = area*(0.0019)
#max_area = area*(0.0048)
min_area = area*(0.0009)
max_area = area*(0.01)

#print(f'{min_area}::{max_area} ')
i = 0

coord = []
for c in contours:
    
    cnt_area = cv2.contourArea(c)
    #if cnt_area > min_area/5 and cnt_area < 10* min_area:
    peri = cv2.arcLength(c, True)
    #approx = cv2.approxPolyDP(c, 0.04 * peri, True)
    approx = cv2.approxPolyDP(c, 0.03 * peri, True)
    x,y,w,h = cv2.boundingRect(c)
    
    ratio = w/h
    if cnt_area > min_area and cnt_area < max_area and len(approx) == 4 :
        #print(len(approx))
        if ratio > 2 or ratio < 0.5:
            continue 
#     if cnt_area > 2250 and cnt_area < 6300 and len(approx) == 4:
        
        i +=1
        #cv2.drawContours(feed, c, -1, (0,255,0),thickness=5)
        #print(f'{area} || {min_area} || {cnt_area} ')
        #print(f'{i} : area= {area} : {cnt_area} : len: {len(approx)}')
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        center_location = (cX,cY)
        boxes[i] = center_location
        coord.append(center_location)
        # draw the contour and center of the shape on the image
        cv2.drawContours(feed, [c], -1, (0, 255, 0), 2)
        cv2.circle(feed, (cX, cY), 30, (255, 255, 0), 4)
        cv2.circle(feed, (cX, cY), 4, (255, 0, 255), -1)
        cv2.putText(feed, str(i), (cX - 10, cY - 10),
    	cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)

In [4]:
#region finding nearest box "left_bottom_key"
left_bottom_location = (0,height)
left_bottom_key = 1
minimum_distace = min_dist(left_bottom_location, boxes[1])
#finding left bottom block
for i in range(1,69,1):
        sepreation = min_dist(left_bottom_location, boxes[i])
        if sepreation < minimum_distace:
            minimum_distace = sepreation
            left_bottom_key = i
#endregion

In [5]:
def nearest_key(ref_point,boxes_left):
    minimum_distace = 1000000 # box length is 70 approx so 500 is much greater then that
    for i in boxes_left.keys():
        sepreation = min_dist(ref_point, boxes_left[i])
        if sepreation < minimum_distace:
            minimum_distace = sepreation
            nearest_key = i
    return nearest_key
radius = 30

In [6]:
global boxes_copy,sorted_keys

In [7]:
def next_key_horizontally(current_key,radius):
    #returns next key
    #add next key to sorted_keys
    
    try:
        x_curr,y_curr = boxes_copy[current_key]
    except:print(f'key: {current_key} failed')
    
    min_horizontal_dist = 5*radius
    #going through all boxes 
    #finding only next key  
    #removing_current_key
    del boxes_copy[current_key]
    
    for i in boxes_copy.keys():
        # find all boxes in the row
        y_deviation = abs(boxes_copy[i][1] - y_curr)
        if  y_deviation < radius :
            # find closest box in horizontal direction
            horizontal_dist = x_curr - boxes_copy[i][0]            
            if  abs(horizontal_dist) < min_horizontal_dist and horizontal_dist < 0 :
                min_horizontal_dist = abs(horizontal_dist)
                next_box_key = i
    #add key and change the current key to newly found key
#     print(f'appended key = {next_box_key}')
    sorted_keys_copy.append(next_box_key)
    next_key = next_box_key
    return next_key

In [8]:
def next_key_vertically(location,radius):
    #returns next key
    #add next key to sorted_keys
    x_curr = location[0]
    y_curr = location[1]
    min_vertical_dist = 5*radius
    #going through all boxes 
    #finding only next key  
    for i in boxes_copy.keys():
        # find all boxes in the row
        x_deviation = abs(boxes_copy[i][0] - x_curr)
        if  x_deviation < radius :
            # find closest box in horizontal direction
            vertical_dist = y_curr - boxes_copy[i][1]


            if  abs(vertical_dist) < min_vertical_dist and vertical_dist > 0 :
                min_vertical_dist = abs(vertical_dist)
                next_box_key = i
    #add key and change the current key to newly found key
#     print(f'appended key = {next_box_key}')
    sorted_keys_copy.append(next_box_key)
    next_key = next_box_key
    return next_key

In [9]:
boxes_copy = boxes.copy()
sorted_keys_copy = sorted_keys.copy()

# flange part of T
left_bottom_key = nearest_key((0,height),boxes_copy)
sorted_keys_copy = [left_bottom_key]
keys_in_range = []
next_box_key = 0
current_key = left_bottom_key

for j in range(17):#last row
        current_key = next_key_horizontally(current_key,radius)

# last-secound-row's first box's key
current_key = next_key_vertically(boxes[left_bottom_key],radius)

for j in range(17):#last row
        current_key = next_key_horizontally(current_key,radius)
print(sorted_keys_copy)


## web part of T
nearest_web_key = nearest_key((0,height),boxes_copy)
current_key = nearest_web_key
sorted_keys_copy.append(nearest_web_key)
print(nearest_web_key)
for i in range(8):
    first_key_of_row = current_key 
    for i in range(3):
        current_key = next_key_horizontally(current_key,radius)
#         print(current_key)
    try:
        current_key = next_key_vertically(boxes[first_key_of_row],radius)
    except:pass
print(sorted_keys_copy)
    
    
        

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 31]
37
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 62, 61, 64, 67, 66, 65, 68]


In [10]:
coord_matrix = np.zeros((10,18,2))
value_matrix = -1*(np.ones((10,18),dtype = np.uint8))
init_key = 0
for i in range(2):
    for j in range(18):
        cX,cY = boxes[sorted_keys_copy[init_key]]
        coord_matrix[9-i,j] = (cX,cY)
        value_matrix[9-i,j] = 2
        init_key +=1
for i in range(7):
    for j in range(7,11):
        cX,cY = boxes[sorted_keys_copy[init_key]]
        coord_matrix[7-i,j] = (cX,cY)
        value_matrix[7-i,j] = 2
        init_key +=1

df = pd.DataFrame(value_matrix)
print(df)

   0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17
0  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
1  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
2  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
3  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
4  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
5  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
6  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
7  -1  -1  -1  -1  -1  -1  -1   2   2   2   2  -1  -1  -1  -1  -1  -1  -1
8   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2
9   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2
